# Getting Started with AWS Converse API


In [1]:
import boto3
import json
from botocore.exceptions import ClientError
from botocore.config import Config

# Optional: extend timeout for image processing (recommended for Nova)
config = Config(connect_timeout=60, read_timeout=3600)

session = boto3.Session()
bedrock = session.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
    config=config
)

### CODE 1 - A basic call to the Converse API

In [2]:
message_list = []

# Your initial user message
initial_message = {
    "role": "user",
    "content": [
        { "text": "How are you today?" }
    ],
}

message_list.append(initial_message)

try:
    response = bedrock.converse(
        modelId="amazon.nova-lite-v1:0",
        messages=message_list,
        inferenceConfig={
            "maxTokens": 2000,
            "temperature": 0.0,
            "topP": 0.9
        }
    )
    response_message = response["output"]["message"]
    print(json.dumps(response_message, indent=4))

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke model. Reason: {e}")
    exit(1)


{
    "role": "assistant",
    "content": [
        {
            "text": "I'm doing well, thank you for asking! As an AI, I don't have feelings, but I'm here and ready to help you with any questions or tasks you have. How can I assist you today?"
        }
    ]
}


###  CODE 2 - Alternating user and assistant messages

####  Alternate between messages from the “user” and “assistant” roles.

In [3]:
message_list.append(response_message)
print(json.dumps(message_list, indent=4))

[
    {
        "role": "user",
        "content": [
            {
                "text": "How are you today?"
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "text": "I'm doing well, thank you for asking! As an AI, I don't have feelings, but I'm here and ready to help you with any questions or tasks you have. How can I assist you today?"
            }
        ]
    }
]


### CODE 3 - Including an image in a message

![Alt Text](image.webp)

In [4]:
# 1️⃣ Initial user text message
message_list.append({
    "role": "user",
    "content": [{"text": "How are you today?"}]
})

# 2️⃣ Image input followed by a prompt
with open("image.webp", "rb") as image_file:
    image_bytes = image_file.read()

image_message = {
    "role": "user",
    "content": [
        {"text": "Image 1:"},
        {
            "image": {
                "format": "webp",
                "source": {"bytes": image_bytes}
            }
        },
        {"text": "Please describe the image."}
    ]
}

message_list.append(image_message)

model_id = "amazon.nova-lite-v1:0"  # or nova‑pro for higher capability

try:
    response = bedrock.converse(
        modelId=model_id,
        messages=message_list,
        inferenceConfig={
            "maxTokens": 2000,
            "temperature": 0.0,
            "topP": 0.9
        }
    )
    response_message = response["output"]["message"]
    print(json.dumps(response_message, indent=4))

    # Continue the conversation
    message_list.append(response_message)

except ClientError as e:
    print("AWS API error:", e.response.get("Error", {}).get("Message"))
except Exception as e:
    print("Unexpected error:", e)


{
    "role": "assistant",
    "content": [
        {
            "text": "The image features a miniature house on a grassy field, surrounded by rocks. The house has a charming design with a red roof and white walls. There are three windows in the front of the house, which is adorned with a red-painted wall, adding a touch of charm and detail. The grass in the area is lush and green, suggesting a healthy and vibrant environment. In the distance, there are some trees that are blurred, indicating a somewhat blurry background that provides depth to the image."
        }
    ]
}


### CODE 4 - Setting a system prompt


In [5]:
# (Assuming you already have previous messages appended here)

# Add the summary request message
summary_message = {
    "role": "user",
    "content": [
        {"text": "Can you please summarize our conversation so far?"}
    ],
}
message_list.append(summary_message)

system_prompts = [
    {"text": "You must respond to all requests in poaragraph."}
]

inference_config = {
    "maxTokens": 2000,
    "temperature": 0.0,
    "topP": 0.9
}

try:
    response = bedrock.converse(
        modelId="amazon.nova-lite-v1:0",  # or nova‑pro, nova‑premier as needed
        messages=message_list,
        system=system_prompts,
        inferenceConfig=inference_config
        # no additionalModelRequestFields unless you need model-specific params like topK
    )
    response_message = response["output"]["message"]
    print(json.dumps(response_message, indent=4))

    # Save assistant response into conversation history
    message_list.append(response_message)
    #print(message_list)

    print("Stop Reason:", response.get("stopReason"))
    print("Usage:", json.dumps(response.get("usage", {}), indent=4))

except ClientError as e:
    print("AWS API error:", e.response.get("Error", {}).get("Message"))
except Exception as e:
    print("Unexpected error:", e)


{
    "role": "assistant",
    "content": [
        {
            "text": "Certainly! Here's a summary of our conversation so far:\n\n1. You asked how I was doing, and I explained that, as an AI, I don't have feelings, but I'm here to help you with any questions or tasks you have.\n2. You shared an image of a miniature house on a grassy field, and I provided a description of the image, mentioning the charming design, red roof, white walls, and lush green grass. I also noted the blurred trees in the distance.\n\nIf you have any more questions or need further assistance, feel free to let me know!"
        }
    ]
}
Stop Reason: end_turn
Usage: {
    "inputTokens": 1509,
    "outputTokens": 122,
    "totalTokens": 1631
}
